In [1]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from numpy import count_nonzero

dir_ = '../data/'
# file_name = 'normalized_filter_track_5_user_100.csv'
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'

In [2]:
predictions = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))

In [3]:
predictions

,uid,tid,rating
0,0.0,0.0,2.493650
1,0.0,6.0,2.333071
2,0.0,8.0,2.713340
3,0.0,9.0,1.949588
4,0.0,12.0,2.477737
...,...,...,...
11741994,219.0,54964.0,1.816855
11741995,219.0,54965.0,2.315346
11741996,219.0,54966.0,1.954432
11741997,219.0,54967.0,2.071366


In [4]:
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

,uid,tid,rating
4,0,0,4.75
10,0,6,1.50
13,0,8,1.50
14,0,9,0.25
20,0,12,0.25
...,...,...,...
1256895,219,54715,0.25
1256896,219,54718,0.25
1256949,219,54900,0.25
1256989,219,54958,0.25


In [5]:
num_user = len(predictions['uid'].unique())
num_user

220

In [6]:
# Personal Recommendation
n=20
top_n_lists = []

for i in tqdm.tqdm_notebook(range(num_user)):
    prediction = predictions[predictions['uid'] == i]
    prediction = prediction.sort_values(by=['rating'],  ascending=False)
    prediction = prediction[:n]
    top_n_list = []
    for _, row in prediction.iterrows():
        top_n_list.append(row[1])
    top_n_lists.append(top_n_list)

In [7]:
evaluation = []
satisfication = 0
for i in tqdm.tqdm_notebook(range(num_user)):
    top_n_list = top_n_lists[i]
    precision = 0
    for j in top_n_list:
        p = test[test['uid'] == i]
        p = p[p['tid']==j]
        if len(p) > 0:
            precision += 1
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/num_user)


0.14977272727272742


In [268]:
#############################################
#                                           #
#          Group Recommendation             #
#                                           #
#############################################

In [269]:
#############################################
#                                           #
#          Group Recommendation (2)         #
#                                           #
#############################################

In [8]:
groups = np.loadtxt(os.path.join(dir_, 'groups_2_' + file_name), delimiter=',')
group_size = len(groups[0])
groups[0]

array([32., 78.])

In [9]:
def max_satisfaction(df):  
    return max(df['rating_u1'], df['rating_u2'])

def least_misery(df):
    return min(df['rating_u1'], df['rating_u2'])

In [10]:
# Prediction list of each group (2)
n=4
top_n_lists_g = []
    
for group in tqdm.tqdm_notebook(groups):
    u1 = predictions[predictions['uid'] == group[0]]
    u2 = predictions[predictions['uid'] == group[1]]

    m = pd.merge(u1, u2, on=['tid'], how='inner', suffixes=['_u1','_u2'])
    m = m[['tid', 'rating_u1','rating_u2']]
    
#   Average
    m['rating'] = (m['rating_u1'] + m['rating_u2'])/2

#   Max Satisfaction
#     m['rating'] = m.apply(max_satisfaction, axis = 1)

#   Least Misery
#     m['rating'] = m.apply(least_misery, axis = 1)
    
    m = m[['tid', 'rating']]
    m = m.sort_values(by=['rating'],  ascending=False)
    m = m[:n]
    top_n_list = []
    for _, row in m.iterrows():
        top_n_list.append(row[0])
    top_n_lists_g.append(top_n_list)

In [26]:
evaluation = []
satisfication = 0
for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    precision = 0
    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]

        s = len(t1) + len(t2)
        if s > 0:
            rating = 3.0
            high_rating = 0
            high_rating_t1 = 0
            high_rating_t2 = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
#             if high_rating > 0:
#                 precision += 1
            precision += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/(num_user//group_size))


0.15251206827110922


In [34]:
# modified satisfaction
satisfication = 0
for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    satisfication_t = 0
    satisfication_u = 0
    high_rating_u1 = 0
    high_rating_u2 = 0
    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]

        s = len(t1) + len(t2)
        if s > 0:
            rating = 4.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u1 += 1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u2 += 1
            satisfication_t += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication_u += math.log(high_rating_u1+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u2+1,10)/math.log(n+1,10)

    satisfication += 0.5 * satisfication_t / n
    satisfication += 0.5 * satisfication_u / len(group)
print(satisfication/(num_user//group_size))


0.1197241849370243


In [210]:
#############################################
#                                           #
#          Group Recommendation (4)         #
#                                           #
#############################################

In [37]:
groups = np.loadtxt(os.path.join(dir_, 'groups_4_' + file_name), delimiter=',')
group_size = len(groups[0])
groups[0]

array([ 16.,  69., 111., 129.])

In [38]:
def max_satisfaction(df):  
    return max(df['rating_u1'], df['rating_u2'], df['rating_u3'], df['rating_u4'])

def least_misery(df):
    return min(df['rating_u1'], df['rating_u2'], df['rating_u3'], df['rating_u4'])

In [39]:
# Prediction list of each group (4)
n=4
top_n_lists_g = []
    
for group in tqdm.tqdm_notebook(groups):
    u1 = predictions[predictions['uid'] == group[0]]
    u2 = predictions[predictions['uid'] == group[1]]
    u3 = predictions[predictions['uid'] == group[2]]
    u4 = predictions[predictions['uid'] == group[3]]
    
    m = pd.merge(u1, u2, on=['tid'], how='inner', suffixes=['_u1','_u2'])
    m = pd.merge(m,  u3, on=['tid'], how='inner', suffixes=['','_u3'])
    m = pd.merge(m,  u4, on=['tid'], how='inner', suffixes=['_u3','_u4'])
    m = m[['tid', 'rating_u1','rating_u2','rating_u3','rating_u4']]
#     Average
    m['rating'] = (m['rating_u1'] + m['rating_u2'] + m['rating_u3'] + m['rating_u4'])/4

#   Max Satisfaction
#     m['rating'] = m.apply(max_satisfaction, axis = 1)

#   Least Misery
#     m['rating'] = m.apply(least_misery, axis = 1)
    
    m = m[['tid', 'rating']]
    m = m.sort_values(by=['rating'],  ascending=False)
    m = m[:n]
    top_n_list = []
    for _, row in m.iterrows():
        top_n_list.append(row[0])
    top_n_lists_g.append(top_n_list)

In [42]:
satisfication = 0
for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    precision = 0
    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4)
        if s > 0:
            rating = 1.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t3) > 0 and t3.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t4) > 0 and t4.iloc[0]['rating'] > rating:
                high_rating +=1
#             if high_rating > 0:
#                 precision += 1
            precision += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication += precision / n
print(satisfication/(num_user//group_size))


0.2489600415656076


In [46]:
# modified satisfaction
satisfication = 0
for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    satisfication_t = 0
    satisfication_u = 0
    high_rating_u1 = 0
    high_rating_u2 = 0
    high_rating_u3 = 0
    high_rating_u4 = 0
    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4)
        if s > 0:
            rating = 4.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u1 += 1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u2 += 1
            if len(t3) > 0 and t3.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u3 += 1
            if len(t4) > 0 and t4.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u4 += 1
            satisfication_t += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication_u += math.log(high_rating_u1+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u2+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u3+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u4+1,10)/math.log(n+1,10)
    satisfication += 0.5 * satisfication_t / n
    satisfication += 0.5 * satisfication_u / len(group)
print(satisfication/(num_user//group_size))


0.14758117494863116


In [121]:
#############################################
#                                           #
#          Group Recommendation (5)         #
#                                           #
#############################################

In [48]:
groups = np.loadtxt(os.path.join(dir_, 'groups_5_' + file_name), delimiter=',')
group_size = len(groups[0])
groups[0]

array([ 11.,  23., 107., 122., 141.])

In [49]:
def max_satisfaction(df):  
    return max(df['rating_u1'], df['rating_u2'], df['rating_u3'], df['rating_u4'], df['rating_u5'])

def least_misery(df):
    return min(df['rating_u1'], df['rating_u2'], df['rating_u3'], df['rating_u4'], df['rating_u5'])

In [50]:
# Prediction list of each group (5)
n=4
top_n_lists_g = []
    
for group in tqdm.tqdm_notebook(groups):
    u1 = predictions[predictions['uid'] == group[0]]
    u2 = predictions[predictions['uid'] == group[1]]
    u3 = predictions[predictions['uid'] == group[2]]
    u4 = predictions[predictions['uid'] == group[3]]
    u5 = predictions[predictions['uid'] == group[4]]

    m = pd.merge(u1, u2, on=['tid'], how='inner', suffixes=['_u1','_u2'])
    m = pd.merge(m,  u3, on=['tid'], how='inner', suffixes=['','_u3'])
    m = pd.merge(m,  u4, on=['tid'], how='inner', suffixes=['','_u4'])
    m = pd.merge(m,  u5, on=['tid'], how='inner', suffixes=['_u3','_u5'])
    m = m[['tid', 'rating_u1','rating_u2','rating_u3','rating_u4','rating_u5']]
#     Average
    m['rating'] = (m['rating_u1'] + m['rating_u2'] + m['rating_u3'] + m['rating_u4'] + m['rating_u5'])/5
    
#     Max Satisfaction
#     m['rating'] = m.apply(max_satisfaction, axis = 1)
    
#     Least Misery
#     m['rating'] = m.apply(least_misery, axis = 1)
    
    m = m[['tid', 'rating']]
    m = m.sort_values(by=['rating'],  ascending=False)
    m = m[:n]
    top_n_list = []
    for _, row in m.iterrows():
        top_n_list.append(row[0])
    top_n_lists_g.append(top_n_list)

In [51]:
evaluation = []
satisfication = 0
for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    precision = 0
    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        t5 = test[test['uid'] == group[4]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        t5 = t5[t5['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5)
        if s > 0:
            rating = 1.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t3) > 0 and t3.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t4) > 0 and t4.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t5) > 0 and t5.iloc[0]['rating'] > rating:
                high_rating +=1
            if high_rating > 0:
                precision += 1
#             precision += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/(num_user//group_size))


0.5965909090909091


In [56]:
# modified satisfaction
satisfication = 0
for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    satisfication_t = 0
    satisfication_u = 0
    high_rating_u1 = 0
    high_rating_u2 = 0
    high_rating_u3 = 0
    high_rating_u4 = 0
    high_rating_u5 = 0

    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        t5 = test[test['uid'] == group[4]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        t5 = t5[t5['tid']==j]

        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5)
        if s > 0:
            rating = 0.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u1 += 1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u2 += 1
            if len(t3) > 0 and t3.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u3 += 1
            if len(t4) > 0 and t4.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u4 += 1
            if len(t5) > 0 and t5.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u5 += 1                
            satisfication_t += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication_u += math.log(high_rating_u1+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u2+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u3+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u4+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u5+1,10)/math.log(n+1,10)
    satisfication += 0.5 * satisfication_t / n
    satisfication += 0.5 * satisfication_u / len(group)
print(satisfication/(num_user//group_size))


0.3328882965764399


In [ ]:
#############################################
#                                           #
#          Group Recommendation (10)        #
#                                           #
#############################################

In [58]:
# np.savetxt(os.path.join(dir_, 'groups_10_' + file_name[:-3] + 'csv'), groups, delimiter=",")
groups = np.loadtxt(os.path.join(dir_, 'groups_10_' + file_name), delimiter=',')
group_size = len(groups[0])
groups[0]

array([ 17.,  58.,  73., 104., 112., 164., 207., 214., 218., 219.])

In [59]:
def max_satisfaction(df):  
    return max(df['rating_u1'], df['rating_u2'], df['rating_u3'], df['rating_u4'], df['rating_u5'], df['rating_u6'], df['rating_u7'], df['rating_u8'], df['rating_u9'], df['rating_u10'])

def least_misery(df):  
    return max(df['rating_u1'], df['rating_u2'], df['rating_u3'], df['rating_u4'], df['rating_u5'], df['rating_u6'], df['rating_u7'], df['rating_u8'], df['rating_u9'], df['rating_u10'])

In [60]:
# Prediction list of each group (10)
n=4
top_n_lists_g = []
    
for group in tqdm.tqdm_notebook(groups):
    u1 = predictions[predictions['uid'] == group[0]]
    u2 = predictions[predictions['uid'] == group[1]]
    u3 = predictions[predictions['uid'] == group[2]]
    u4 = predictions[predictions['uid'] == group[3]]
    u5 = predictions[predictions['uid'] == group[4]]
    u6 = predictions[predictions['uid'] == group[5]]
    u7 = predictions[predictions['uid'] == group[6]]
    u8 = predictions[predictions['uid'] == group[7]]
    u9 = predictions[predictions['uid'] == group[8]]
    u10 = predictions[predictions['uid'] == group[9]]

    m = pd.merge(u1, u2,  on=['tid'], how='inner', suffixes=['_u1','_u2'])
    m = pd.merge(m,  u3,  on=['tid'], how='inner', suffixes=['','_u3'])
    m = pd.merge(m,  u4,  on=['tid'], how='inner', suffixes=['','_u4'])
    m = pd.merge(m,  u5,  on=['tid'], how='inner', suffixes=['','_u5'])
    m = pd.merge(m,  u6,  on=['tid'], how='inner', suffixes=['','_u6'])
    m = pd.merge(m,  u7,  on=['tid'], how='inner', suffixes=['','_u7'])
    m = pd.merge(m,  u8,  on=['tid'], how='inner', suffixes=['','_u8'])
    m = pd.merge(m,  u9,  on=['tid'], how='inner', suffixes=['','_u9'])
    m = pd.merge(m,  u10, on=['tid'], how='inner', suffixes=['_u3','_u10'])
    m = m[['tid', 'rating_u1','rating_u2','rating_u3','rating_u4','rating_u5', 'rating_u6','rating_u7','rating_u8','rating_u9','rating_u10']]

#     Average
    m['rating'] = (m['rating_u1'] + m['rating_u2'] + m['rating_u3'] + m['rating_u4'] + m['rating_u5'] + m['rating_u6'] + m['rating_u7'] + m['rating_u8'] + m['rating_u9'] + + m['rating_u10'])/10
    
#     Max Satisfaction
#     m['rating'] = m.apply(max_satisfaction, axis = 1)
    
#     Least Misery
#     m['rating'] = m.apply(least_misery, axis = 1)
    
    m = m[['tid', 'rating']]
    m = m.sort_values(by=['rating'],  ascending=False)
    m = m[:n]
    top_n_list = []
    for _, row in m.iterrows():
        top_n_list.append(row[0])
    top_n_lists_g.append(top_n_list)

In [162]:
evaluation = []
satisfication = 0
for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    precision = 0
    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        t5 = test[test['uid'] == group[4]]
        t6 = test[test['uid'] == group[5]]
        t7 = test[test['uid'] == group[6]]
        t8 = test[test['uid'] == group[7]]
        t9 = test[test['uid'] == group[8]]
        t10 = test[test['uid'] == group[9]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        t5 = t5[t5['tid']==j]
        t6 = t6[t6['tid']==j]
        t7 = t7[t7['tid']==j]
        t8 = t8[t8['tid']==j]
        t9 = t9[t9['tid']==j]
        t10 = t10[t10['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5) + len(t6) + len(t7) + len(t8) + len(t9) + len(t10)
        if s > 0:
            rating = 1.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t3) > 0 and t3.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t4) > 0 and t4.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t5) > 0 and t5.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t6) > 0 and t6.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t7) > 0 and t7.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t8) > 0 and t8.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t9) > 0 and t9.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t10) > 0 and t10.iloc[0]['rating'] > rating:
                high_rating +=1
            if high_rating > 0:
                precision += 1
#             precision += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/(num_user//group_size))


0.7159090909090909


In [65]:
# modified satisfaction
satisfication = 0
for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    satisfication_t = 0
    satisfication_u = 0
    high_rating_u1 = 0
    high_rating_u2 = 0
    high_rating_u3 = 0
    high_rating_u4 = 0
    high_rating_u5 = 0
    high_rating_u6 = 0
    high_rating_u7 = 0
    high_rating_u8 = 0
    high_rating_u9 = 0
    high_rating_u10 = 0

    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        t5 = test[test['uid'] == group[4]]
        t6 = test[test['uid'] == group[5]]
        t7 = test[test['uid'] == group[6]]
        t8 = test[test['uid'] == group[7]]
        t9 = test[test['uid'] == group[8]]
        t10 = test[test['uid'] == group[9]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        t5 = t5[t5['tid']==j]
        t6 = t6[t6['tid']==j]
        t7 = t7[t7['tid']==j]
        t8 = t8[t8['tid']==j]
        t9 = t9[t9['tid']==j]
        t10 = t10[t10['tid']==j]
        
        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5) + len(t6) + len(t7) + len(t8) + len(t9) + len(t10)
        if s > 0:
            rating = 4.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u1 += 1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u2 += 1
            if len(t3) > 0 and t3.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u3 += 1
            if len(t4) > 0 and t4.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u4 += 1
            if len(t5) > 0 and t5.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u5 += 1  
            if len(t6) > 0 and t6.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u6 += 1  
            if len(t7) > 0 and t7.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u7 += 1  
            if len(t8) > 0 and t8.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u8 += 1  
            if len(t9) > 0 and t9.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u9 += 1  
            if len(t10) > 0 and t10.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u10 += 1  
            satisfication_t += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication_u += math.log(high_rating_u1+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u2+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u3+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u4+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u5+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u6+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u7+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u8+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u9+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u10+1,10)/math.log(n+1,10)
    satisfication += 0.5 * satisfication_t / n
    satisfication += 0.5 * satisfication_u / len(group)
print(satisfication/(num_user//group_size))


0.1695292296740876


In [326]:
groups = []
group_size = 10

u_del = []
num_u_del = num_user - (num_user//group_size)*group_size

for i in range(num_u_del):
    u = random.random()*num_user//1
    u_del.append(u)

for i in range(num_user//group_size):
    groups.append([])
    
for i in range(num_user):
    b = True
    
    if i in u_del:
        b = False
        
    while b:
        g = random.random()*(num_user/group_size)//1
        if int(g) < len(groups):
            group = groups[int(g)]
            if len(group) < group_size:
                group.append(i)
                b = False
groups

[[28, 51, 73, 87, 99, 113, 117, 118, 136, 137],
 [10, 13, 20, 21, 39, 50, 110, 119, 128, 177],
 [16, 32, 58, 77, 88, 127, 159, 179, 183, 189],
 [2, 3, 7, 29, 30, 34, 36, 47, 57, 66],
 [6, 68, 76, 98, 103, 120, 125, 139, 157, 172],
 [44, 75, 121, 173, 196, 203, 206, 207, 209, 210],
 [0, 9, 65, 67, 70, 104, 106, 144, 150, 156],
 [4, 27, 40, 56, 74, 83, 91, 97, 100, 105],
 [17, 18, 24, 37, 46, 146, 149, 175, 176, 192],
 [22, 60, 80, 81, 131, 133, 135, 138, 155, 164],
 [25, 26, 33, 49, 130, 171, 184, 194, 197, 200],
 [14, 41, 52, 62, 63, 64, 84, 86, 102, 108],
 [61, 71, 90, 116, 129, 132, 165, 185, 190, 195],
 [38, 45, 85, 93, 140, 147, 161, 181, 191, 198],
 [12, 42, 89, 92, 96, 107, 142, 188, 205, 208],
 [43, 53, 101, 112, 126, 152, 163, 182, 186, 187],
 [1, 19, 23, 48, 78, 115, 123, 145, 148, 167],
 [35, 54, 55, 69, 109, 124, 141, 153, 158, 170],
 [15, 31, 82, 95, 151, 193, 199, 201, 202, 204],
 [5, 79, 134, 143, 160, 162, 166, 168, 178, 180],
 [8, 11, 59, 94, 111, 114, 122, 154, 169, 17

In [327]:
# check if duplicate 

users = []
for group in groups:
    for u in group:
        users.append(u)
users.sort(reverse=True)

n = num_user - 1 
for i in users:
    if i-n != 0:
        print(i, 'False')
        n-=1
    n-=1

71 False


In [328]:
np.savetxt(os.path.join(dir_, 'groups_' + str(group_size) + '_' + file_name[:-3] + 'csv'), groups, delimiter=",")